In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
# Assuming your files are located in the following paths:
models_path = '/content/drive/MyDrive/HW_5_CS/models.py'

# Import the files
import sys
sys.path.append('/content/drive/MyDrive/MyModelsAndUtils')

# Now you can import your models and utils as if they were in the Colab environment
from models import ModelA, ModelB


In [ ]:
import os
import shutil
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from itertools import product
from scipy.signal import find_peaks
from scipy.integrate import solve_ivp

from models import *



save_figures = True
save_dir = './Report//figs'
questions_number = '1_2_3'
if save_figures:
    if not os.path.exists(save_dir):
        os.makedirs(save_dir, exist_ok=True)

Part_1

In [ ]:
input_currents = [
    lambda t: 20 if (t>=60) else 40 if (t>=20 and t<=20.5) else 0.0,
]

t_start = 0
t_end = 100
dt = 0.01
t = np.arange(t_start, t_end, dt)

attributes_to_change = ['C',]

k = 0

for attribute in attributes_to_change:
    for input_current in input_currents:
        for percentage in np.arange(0.5,2.5,0.5):
            model = HodgkinHuxleyModel()
            setattr(model, attribute, getattr(model, attribute) * percentage)

            sol = solve_ivp(
                lambda t, Y: model(Y, I=input_current(t)),
                t_span=[t_start, t_end],
                y0=model.get_initial_state(),
                t_eval=t,
                vectorized=True,
                method = 'RK23',
            )

            V, n, m, h = sol.y
            I_Na, I_K, I_L = model.get_currents(*sol.y)

            fig, ax = plt.subplots(nrows=2, ncols=2, sharex=True, figsize=(12.8, 4.8))
            ax[0,0].plot(sol.t, np.vectorize(input_current)(sol.t))
            ax[0,0].set_ylim([-2,32])
            ax[0,0].set_xlabel('$t\;[m\mathrm{s}]$')
            ax[0,0].set_ylabel('$I\;[\mu\mathrm{A}/c\mathrm{m}^2]$')
            ax[0,0].set_title('Input Current')
            ax[0,0].grid()
            ax[0,1].plot(sol.t, V)
            ax[0,1].set_ylim([-90,50])
            ax[0,1].set_xlabel('$t\;[m\mathrm{s}]$')
            ax[0,1].set_ylabel('$V\;[m\mathrm{V}]$')
            ax[0,1].set_title('Membrane Voltage')
            ax[0,1].grid()
            ax[1,0].plot(sol.t, I_Na, label='$I_\mathrm{Na}$')
            ax[1,0].plot(sol.t, I_K, label='$I_\mathrm{K}$')
            ax[1,0].plot(sol.t, I_L, label='$I_\mathrm{L}$')
            ax[1,0].set_xlabel('$t\;[m\mathrm{s}]$')
            ax[1,0].set_ylabel('$I\;[\mu\mathrm{A}/c\mathrm{m}^2]$')
            ax[1,0].set_title('Ion and leak currents')
            ax[1,0].legend(loc='lower right')
            ax[1,0].grid()
            ax[1,1].plot(sol.t, n, label='$n$')
            ax[1,1].plot(sol.t, m, label='$m$')
            ax[1,1].plot(sol.t, h, label='$h$')
            ax[1,1].set_xlabel('$t\;[m\mathrm{s}]$')
            ax[1,1].set_title('Gating Variables')
            ax[1,1].legend(loc='lower right')
            ax[1,1].grid()
            fig.suptitle(f'Hodgkin-Huxley Model Response with ${attribute if attribute != "g_Na" else "g_{Na}"}=' + f'{getattr(model, attribute):.2f}$')
            plt.tight_layout()
            if save_figures:
                plt.savefig(f'{save_dir}/{questions_number}_1_{k}.png')
                k += 1

Part_2

In [ ]:
attributes_to_change = ['C', 'g_Na', 'g_K', 'g_L']

k = 0

t_start = 0
t_end = 20
dt = 0.01
t = np.arange(t_start, t_end, dt)

percentages_to_test = np.arange(0.64,1.5,0.05)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12.8, 4.8))

for attribute in (tqdm_loader := tqdm(attributes_to_change, unit='parameter')):

    threshold_impulse_current = np.zeros_like(percentages_to_test)

    last_threshold = None

    for i, percentage in enumerate(percentages_to_test):
        if last_threshold is None:
            currents_to_test = np.arange(1,30,0.5)
        else:
            currents_to_test = np.arange(last_threshold-5, last_threshold+5.1, 0.4)

        for I in currents_to_test:
            if I <= 0:
                continue

            input_current = lambda t: I if (t>=0.5 and t<=1) else 0.0

            model = HodgkinHuxleyModel()
            setattr(model, attribute, getattr(model, attribute) * percentage)

            tqdm_loader.set_description(f'Calculating impulse current thresholds for {attribute} = {getattr(model, attribute):.2f}')

            sol = solve_ivp(
                lambda t, Y: model(Y, I=input_current(t)),
                t_span=[t_start, t_end],
                y0=model.get_initial_state(),
                t_eval=t,
                vectorized=True,
                method = 'RK23',
            )

            model.set_initial_state(*sol.y[:,-1])

            sol = solve_ivp(
                lambda t, Y: model(Y, I=input_current(t)),
                t_span=[t_start, t_end],
                y0=model.get_initial_state(),
                t_eval=t,
                vectorized=True,
                method = 'RK23',
            )

            V, n, m, h = sol.y

            if np.any(V > 0):
                threshold_impulse_current[i] = I
                last_threshold = I
                break

    ax[0].plot(percentages_to_test * 100, threshold_impulse_current, label=f'${attribute if attribute != "g_Na" else "g_{Na}"}$')
ax[0].set_xlabel('Percentage of change')
ax[0].set_ylabel('$I_\mathrm{threshold}\;[\mu\mathrm{A}/c\mathrm{m}^2]$')
ax[0].set_title('Effect of changing parameters on cell excitability, impulse response')
ax[0].legend()
ax[0].grid()

for attribute in (tqdm_loader := tqdm(attributes_to_change, unit='parameter')):

    threshold_impulse_current = np.zeros_like(percentages_to_test)

    last_threshold = None

    for i, percentage in enumerate(percentages_to_test):
        if last_threshold is None:
            currents_to_test = np.arange(1,30,0.5)
        else:
            currents_to_test = np.arange(last_threshold-5, last_threshold+5.1, 0.4)

        for I in currents_to_test:
            if I <= 0:
                continue

            input_current = lambda t: I if t>=0.5 else 0.0

            model = HodgkinHuxleyModel()
            setattr(model, attribute, getattr(model, attribute) * percentage)

            tqdm_loader.set_description(f'Calculating step current thresholds for {attribute} = {getattr(model, attribute):.2f}')

            sol = solve_ivp(
                lambda t, Y: model(Y, I=input_current(t)),
                t_span=[t_start, t_end],
                y0=model.get_initial_state(),
                t_eval=t,
                vectorized=True,
                method = 'RK23',
            )

            model.set_initial_state(*sol.y[:,-1])

            sol = solve_ivp(
                lambda t, Y: model(Y, I=input_current(t)),
                t_span=[t_start, t_end],
                y0=model.get_initial_state(),
                t_eval=t,
                vectorized=True,
                method = 'RK23',
            )

            V, n, m, h = sol.y

            if np.any(V > 0):
                threshold_impulse_current[i] = I
                last_threshold = I
                break

    ax[1].plot(percentages_to_test * 100, threshold_impulse_current, label=f'${attribute if attribute != "g_Na" else "g_{Na}"}$')
ax[1].set_xlabel('Percentage of change')
ax[1].set_ylabel('$I_\mathrm{threshold}\;[\mu\mathrm{A}/c\mathrm{m}^2]$')
ax[1].set_title('Effect of changing parameters on cell excitability, step response')
ax[1].legend()
ax[1].grid()
plt.tight_layout()
if save_figures:
    plt.savefig(f'{save_dir}/{questions_number}_2_{k}.png')

plt.show()